In [ ]:
!pip install torchvision datasets peft trl bitsandbytes accelerate

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser, 
    TrainingArguments, 
    pipeline, 
    logging,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
base_model = "beomi/llama-2-ko-7b"
new_model = "llama-2-7b-cloudformation-v5-2" 

In [ ]:
# LoRA param
lora_r = 64   
lora_alpha = 16  
lora_dropout = 0.1

In [ ]:
# 4-bit precision
use_4bit = True 
bnb_4bit_compute_dtype = "float16" 
bnb_4bit_quant_type = "nf4" 
use_nested_quant = False 

In [ ]:
# training param
output_dir = "./results" 
num_train_epochs = 30
fp16 = False   
bf16 = True   
per_device_train_batch_size = 1 
per_device_eval_batch_size = 1  
gradient_accumulation_steps = 8
gradient_checkpointing = True  
max_grad_norm = 0.3  
# learning_rate = 2e-6 
weight_decay = 0.001 
optim = "paged_adamw_32bit"  
lr_scheduler_type = "cosine"   
max_steps = -1 
warmup_ratio = 0.03  
group_by_length = True   
save_steps = 0  
logging_steps = 1

In [ ]:
max_seq_length = 1500
packing = False  
device_map = {"": 0}  

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
data_list = os.listdir("./final5")
file_path = []
for i in data_list:
    file_path.append("./final5/" + i)
    
dataset = load_dataset("text", data_files=file_path)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.add_special_tokens = False
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    # learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model) 

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map
)

In [ ]:
model = PeftModel.from_pretrained(base_model, "llama-2-7b-cloudformation-v6")
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained("beomi/llama-2-ko-7b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  
tokenizer.padding_side = "right"  

In [ ]:
from huggingface_hub import interpreter_login

interpreter_login()

In [ ]:
model.push_to_hub("winF/llama-2-7b-cloudformation", use_temp_dir=False)
tokenizer.push_to_hub("winF/llama-2-7b-cloudformation", use_temp_dir=False)